<h1>Lenstra-Lenstra-Lovász</h1>
<h2>Konstruktion einer Gitterbasis</h2>

In [31]:
n = 10; m = 5;

In [32]:
B=random_matrix(ZZ,m,n,x=-10,y=11)
B

[-7 -8 -7 -1 -6  9  1  2  2 -4]
[ 7 -3  3 -2  2  8  9 10 -3 -2]
[ 6 -6  2 -6  7  4 10  6  2  7]
[-3 -9 10 -7  4 -3  7  3 -1 -8]
[ 7  9 -6 -2  2  4  9  6 -7 -1]

Dies definiert ein Gitter im $\mathbb{R}^n$ (erzeugt von den Zeilen).

In [33]:
def random_elementary_matrix():
    #generate 2 distinct numbers a and b
    a=randint(0,m-1)
    b=randint(0,m-2)
    if b==a:
        b=m-1
    return elementary_matrix(m,row1=a,row2=b,scale=ZZ(randint(-10,10)))

In [34]:
BB=B
for i in range(40):
    BB=random_elementary_matrix()*BB
BB

[   261039946   -251310829     23776251    171600424   -110118299    521133664     27497004    335385757    102344877    158921153]
[  1280124843  -1232411119    116596448    841517747   -540013093   2555607665    134844729   1644711793    501891793    779339259]
[ -8546736696   8228225962   -778471942  -5618397550   3605402809 -17062512832   -900270300 -10980907672  -3350903098  -5203265223]
[  2844123364  -2738131448    259054411   1869651066  -1199780738   5677943798    299585716   3654149754   1115090056   1731506225]
[-15962462884  15367544417  -1453910600 -10493284427   6733682131 -31867071805  -1681423226 -20508674708  -6258345677  -9717953821]

Die Zeilen von BB spannen immer noch das gleiche Gitter auf wie die Zeilen von B.

<h2>Weitere Funktionen</h2>
Einige weitere einfache Sage-funktionen, die nützlich sein könnten.

Skalarprodukt $v^Tw$ von Vektoren:

In [7]:
v=vector([1,2,3])
w=vector([7,2,0])
v*w

11

Einträge vertauschen:

In [8]:
(v[1],v[2])=(v[2],v[1])
v

(1, 3, 2)

Runden zur nächsten ganzen Zahl:

In [9]:
(8.62).round()

9

<h2>Aufgabe</h2>
Implementieren Sie LLL mit delta=3/4 und wenden Sie den Algorithmus auf das von den Zeilen von BB aufgespannte Gitter an. Bekommt man das urspruengliche B zurueck?

In [10]:
def gram_schmidt(base):
    """Calculates the gram-schmidt base of the base defined by the rows of the given matrix"""
    
    gs_base = Matrix(base.base_ring(), *base.dimensions())
    
    for i, base_vector in enumerate(base):
        gs_base_vector = base_vector
        
        # SUM (for j = 0 to i - 1)
        for j in range(0, i): # Mind that range() is a semi-open interval, i is exclusive.
            # In Sage, * for vectors denotes the inner product - which is what we want
            scalar = (base_vector* gs_base[j]) / (gs_base[j] *gs_base[j])
            gs_base_vector -= scalar * base_vector

        gs_base[i] = gs_base_vector
        
    return gs_base

In [11]:
# Every week my 'tests' are straying farther from the light.
for i in range(100):
    try:
        x, y = randint(2, 15), randint(2, 15)
        m = random_matrix(QQ, x, y)
        my_gs = gram_schmidt(m)
        sage_gs, _ = m.gram_schmidt()
        
        if my_gs.row_space() != sage_gs.row_space():
            print(f"Sagemath GS base and custom implementation did not produce equivalent base for input {m}")
            assert(False)
            
    except ZeroDivisionError:
        # I'm sure there'd be a way to construct matrices such that we do not produce 
        # divisions by zero in GS reduction, but that would require thinking.
        print("Skipping due to division by zero")
        continue


Skipping due to division by zero
Skipping due to division by zero
Skipping due to division by zero
Skipping due to division by zero
Skipping due to division by zero


In [190]:
def lll(base, delta=3/4):
    # Base vectors are m rows of base matrix
    m = base.dimensions()[0]
    lll_base = copy(base)
    
    done = False
      
    while not done:
        # GS will not work over the integers, so we'll use double-precision floats
        gs_base = gram_schmidt(lll_base.change_ring(RDF))
        # gs_base, _ = lll_base.change_ring(RDF).gram_schmidt()
        
        for i in range(1, m): # [1, m-1]
            for j in range(i-1, -1, -1): # [i-1, 0]
                tmp = (lll_base[i] * gs_base[j]) / (gs_base[j] * gs_base[j])
                lll_base[i] -= round(tmp) * lll_base[j]
        
        done = True
        for i in range(0, m-1): # [0, m-2]
            mu = (lll_base[i + 1] * gs_base[i]) / (gs_base[i] * gs_base[i])
            
            threshold = ((mu * gs_base[i] + gs_base[i + 1]).norm())^2
            if delta * (gs_base[i].norm())^2 > threshold:
                done = False
                break
                
        if not done:
            lll_base[i], lll_base[i + 1] = lll_base[i + 1], lll_base[i]
            
    
    return lll_base

In [184]:
A = matrix(ZZ, [[1, -1, 3], [1, 0, 5], [1, 2, 6]])
A_lll = A.LLL()
Mine = lll(A)
Wiki = matrix(ZZ, [[0, 1, -1], [1, 0, 0], [0, 1, 2]])
Wolfram = matrix(ZZ, [[0, -1, 1], [1, -1, 0], [1, 1, 1]])

print(A, A_lll, Wiki, Wolfram, sep="\n----\n")
print("A vs A.LLL", A.row_space() == A_lll.row_space())
print("A vs Mine", A.row_space() == Mine.row_space())
print("A vs Wiki", A.row_space() == Wiki.row_space())
print("A vs Wolfram", A.row_space() == Wolfram.row_space())

(1, 0, 5) (-0.3015113445777635, 0.30151134457776363, -0.9045340337332909)
(1, 2, 6) (0.2752409412815895, -0.8807710121010887, -0.385337317794226)
(25, -18, 86) (-0.3015113445777635, 0.30151134457776363, -0.9045340337332909)
[ 1 -1  3]
[ 1  0  5]
[ 1  2  6]
----
[ 1 -1  0]
[-1  0  1]
[ 1  1  1]
----
[ 0  1 -1]
[ 1  0  0]
[ 0  1  2]
----
[ 0 -1  1]
[ 1 -1  0]
[ 1  1  1]
A vs A.LLL True
A vs Mine True
A vs Wiki False
A vs Wolfram True


In [192]:
lll_bb = lll(BB, delta=0.75)

print(f"LLL reduced base in custom implementation:\n{lll_bb}")
print(f"Sagemath implementation:\n{BB.LLL()}")

LLL reduced base in custom implementation:
[   -9602    11018     -243    -6758     5068   -21778    -1134   -13570    -4382    -5933]
[  -28965   -17688    -8674    -6716    -9301    -2702    -6385   -12341     8006   -11159]
[  -18764     5867   -36464   -11862   -12868    15996    18214     4701   -13559   -23453]
[  -30838   -18774   -23343    -8369   -16275    13539     2186    -3735     3650   -17751]
[ -141509   535749 -1002717  -289989  -224375   452041   677705   297149  -583435  -575950]
Sagemath implementation:
[ 1  3  1  4 -5  4 -1  4 -5 -9]
[-3  3  1 -7  4 -7  7 -1 -5 -7]
[ 0 12 -9  0  0 -4  0 -4 -4  1]
[ 7  9 -6 -2  2  4  9  6 -7 -1]
[-7 -8 -7 -1 -6  9  1  2  2 -4]


In [137]:
print(B.row_space() == BB.row_space())
print(B.row_space() == BB.LLL().row_space())
print(B.row_space() == lll_bb.row_space())

True
True
True


In [180]:
BB.gram_schmidt?